In [1]:
import scipy as sp
import numpy as np
from numbers import Number

In [ ]:
from numbers import Number

def shipping_analysis(A, b, M, kra=True, eq=True, flag=True):
    if not (isinstance(A, np.ndarray) and len(A.shape) == 2
            and isinstance(b, np.ndarray) and len(b.shape) == 1
            and isinstance(M, Number)):
        raise TypeError("A has to be a 2D numpy array, b a 1D numpy array"
                        "and M a number.")
    
    if kra:
        A_w = A
        b_w = b
        C = np.array([[1, 1, 0, 0, 0, 0],
                      [-1, 0, 1, 0, 0, 1],
                      [0, -1, 0, 1, 1, 0]])
    else:
        A_w = A[:-1, :-1]
        b_w = b[:-1]
        C = np.array([[1, 1, 0, 0, 0],
                      [-1, 0, 1, 0, 0],
                      [0, -1, 0, 1, 1]])
    d = np.array([M, 0, 0])
    if not eq:
        A_w *= 2
    A_1 = np.linalg.inv(A_w)
    un = -A_1 @ b_w
    constrained = un - ((A_1 @ C.T) @ (np.linalg.inv(C @ A_1 @ C.T)) @ (C @ un - d))
    if flag:
        if eq:
            text = "Equilibrium Analysis"
        else:
            text = "Social Optimum Analysis"
        if kra:
            text += " (With Kra Canal)"
        else:
            text += " (Without Kra Canal)"
        print(text)
        print(f"Optimal Solution: = {constrained}")
    return constrained
A = np.diag([0.1, 2, 3, 4, 5, 6])
b = np.array([0.1, 3, 4, 5, 6, 7])
M = 100
x = shipping_analysis(A, b, M, kra=False, eq=True, flag=True)

Equilibrium Analysis (Without Kra Canal)
Optimal solution: [58.25644894 41.74355106 58.25644894 23.30197468 18.44157639]
Objective value: 9530.370333839166


In [22]:
def route_check(x):
    if len(x) == 6:
        A_w = A
        b_w = b
        r = np.array([[1, 0, 0, 0, 0, 1],
                      [1, 0, 1, 0, 0, 0],
                      [0, 1, 0, 1, 0, 0],
                      [0, 1, 0, 0, 1, 0]])
    elif len(x) == 5:
        A_w = A[:-1, :-1]
        b_w = b[:-1]
        r = np.array([[1, 0, 1, 0, 0],
                      [0, 1, 0, 1, 0],
                      [0, 1, 0, 0, 1]])
    else:
        raise ValueError('x needs to be 5 or 6 dimensional.')
    return r @ (A_w @ x + b_w)

def average_cost(x):
    if len(x) == 6:
        A_w = A
        b_w = b
    elif len(x) == 5:
        A_w = A[:-1, :-1]
        b_w = b[:-1]
    return (x.T @ A_w @ x + b_w .T @ x) / M
cost = average_cost(x)

In [23]:
def POA(x_bar, x_tilde):
    return average_cost(x_bar)/average_cost(x_tilde)

POA(shipping_analysis(A, b, M, kra=False, eq=True, flag=False),
    shipping_analysis(A, b, M, kra=False, eq=False, flag=False))

np.float64(1.000036396113496)